In [1]:
!python --version

Python 3.9.1


In [2]:
import langchain
langchain.__version__


'0.3.23'

In [3]:
### Initialize langchain agents and tools
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
load_dotenv()
import os
open_api_key = os.getenv("OPENAI_API_KEY")


In [4]:
!pip install -U duckduckgo-search

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached primp-0.14.0-cp38-abi3-win_amd64.whl.metadata (13 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.8 MB 4.2 MB/s eta 0:00:01
   --------------------------- ------------ 2.6/3.8 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 7.8 MB/s eta 0:00:00
Using cached primp-0.14.0-cp38-abi3-win_amd64.whl (3.1 MB)
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
search_tool = Tool(
    name= "DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches such as latest news, entertainement movies etc using DuckDuckGo"
)
llm = OpenAI(temperature =0.5)




C:\Users\suhas\AppData\Local\Temp\ipykernel_323752\1085996933.py:7: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature =0.5)


In [6]:
!pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11787 sha256=fc718dfe7db97413dc02438a0de28ffd77d2d90a4f8f68e80dc7cea2dea601e1
  Stored in directory: c:\users\suhas\appdata\local\pip\cache\wheels\c2\46\f4\caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
## Initialize the Wikipedia 
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
wikipedia_tool= Tool(
    name= "Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to perform wikipedia searches for information "
)

tools=[wikipedia_tool]


In [8]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")

Database created and populated with sample data.


In [9]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///example.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sqllite_tool = toolkit.get_tools()
tools = sqllite_tool


In [10]:
agent = initialize_agent(tools = tools, 
                         llm=llm,
                         agent="zero-shot-react-description",
                        verbose=True
                        )
query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("response: ", response1)

C:\Users\suhas\AppData\Local\Temp\ipykernel_323752\1771633576.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools = tools,
C:\Users\suhas\AppData\Local\Temp\ipykernel_323752\1771633576.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run(query1)




> Entering new AgentExecutor chain...
 I need to query the database to get the number of units of Product A in stock.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM products WHERE name='Product A' AND stock > 0
Observation: Error: (sqlite3.OperationalError) no such table: products
[SQL: SELECT COUNT(*) FROM products WHERE name='Product A' AND stock > 0]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: I need to use sql_db_list_tables to check if the 'products' table exists.
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: I need to use sql_db_schema to get the schema and sample rows for the 'products' table.
Action: sql_db_schema
Action Input: products
Observation: Error: table_names {'products'} not found in database
Thought: I need to use sql_db_query_checker to check the query before executing it.
Action: sql_db_query_checker
Action Input: SELECT COUNT(*) FROM inventory WHERE name='Product A' AND stock > 0
Observation:  SELECT

In [11]:
!pip install arxiv

  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=2ddc93d55711e4769eb9f6b15d7cb11d67a58773d665039944d82bbe45e79880
  Stored in directory: c:\users\suhas\appdata\local\pip\cache\wheels\65\7a\a7\78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k
  Attempting uninstall: requests
    Found existing installation: 

    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain.agents import load_tools
arxiv_tool = Tool(
    name="Arxiv Search",
    func= ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)





In [13]:
##tools = load_tools(["arxiv_tool", "wikipedia_tool", "search_tool", "sqllite_tool"], llm=llm)
tools = [arxiv_tool, search_tool]

In [14]:
agent = initialize_agent(tools, 
                         llm,
                         agent="zero-shot-react-description",
                        verbose=True,
                        handle_parsing_errors=True
                        )


In [15]:
##query1 = "Search for research papers on Diffusion Models in 2024."
##query1 = "Find recent papers on Generative AI and what is the latest movie trending in Charlotte USA this week"
query1 = "In which city was Facebook office started and What are some of the recent papers on Generative AI"
response1 = agent.run(query1)
print("response: ", response1)



> Entering new AgentExecutor chain...
 I should use the tools available to search for the city where Facebook office was started and recent papers on Generative AI
Action: Arxiv Search
Action Input: 'Facebook office location' and 'Generative AI'
Observation: Published: 2024-03-19
Title: How Spammers and Scammers Leverage AI-Generated Images on Facebook for Audience Growth
Authors: Renee DiResta, Josh A. Goldstein
Summary: Much of the research and discourse on risks from artificial intelligence (AI)
image generators, such as DALL-E and Midjourney, has centered around whether
they could be used to inject false information into political discourse. We
show that spammers and scammers - seemingly motivated by profit or clout, not
ideology - are already using AI-generated images to gain significant traction
on Facebook. At times, the Facebook Feed is recommending unlabeled AI-generated
images to users who neither follow the Pages posting the images nor realize
that the images are AI-genera